# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.

%load_ext dotenv
%dotenv ../01_materials/labs/.env


In [2]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

# Write your code below.
#load the env variable 'PRICE DATA'
price_data_path =os.getenv('PRICE_DATA')
print(f"PRICE_DATA_path:{price_data_path}")



PRICE_DATA_path:../../05_src/data/prices/


In [7]:
parquet_files = glob(os.path.join(price_data_path, '**','*.parquet'), recursive = True)
print(f"Parquet files: {parquet_files}")

Parquet files: ['../../05_src/data/prices/AMZN/AMZN_2023.parquet', '../../05_src/data/prices/AMZN/AMZN_2015.parquet', '../../05_src/data/prices/AMZN/AMZN_2014.parquet', '../../05_src/data/prices/AMZN/AMZN_2022.parquet', '../../05_src/data/prices/AMZN/AMZN_2016.parquet', '../../05_src/data/prices/AMZN/AMZN_2020.parquet', '../../05_src/data/prices/AMZN/AMZN_2021.parquet', '../../05_src/data/prices/AMZN/AMZN_2017.parquet', '../../05_src/data/prices/AMZN/AMZN_2024.parquet', '../../05_src/data/prices/AMZN/AMZN_2013.parquet', '../../05_src/data/prices/AMZN/AMZN_2018.parquet', '../../05_src/data/prices/AMZN/AMZN_2019.parquet', '../../05_src/data/prices/RCL/RCL_2019.parquet', '../../05_src/data/prices/RCL/RCL_2018.parquet', '../../05_src/data/prices/RCL/RCL_2013.parquet', '../../05_src/data/prices/RCL/RCL_2024.parquet', '../../05_src/data/prices/RCL/RCL_2021.parquet', '../../05_src/data/prices/RCL/RCL_2017.parquet', '../../05_src/data/prices/RCL/RCL_2016.parquet', '../../05_src/data/prices/RCL

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [9]:
# Write your code below.

print(f"Columns in the DataFrame: {ddf.columns}")

Columns in the DataFrame: Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'ticker', 'year', 'Close_lag'],
      dtype='object')


In [10]:
if parquet_files:
    # Load the parquet files using Dask
    ddf = dd.read_parquet(parquet_files)

    # Print column names to diagnose the issue
    print(f"Columns in the DataFrame: {ddf.columns}")

    # Add lags for the Close variable if it exists
    if 'Close' in ddf.columns:
        ddf['Close_lag'] = ddf['Close'].shift(1)
        # Add returns based on Close
        ddf['returns'] = (ddf['Close'] / ddf['Close_lag']) - 1
    else:
        print("Column 'Close' not found in the DataFrame")

Columns in the DataFrame: Index(['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Dividends',
       'Stock Splits', 'ticker', 'year'],
      dtype='object')


In [12]:
# Add the hi_lo_range: this is the day's High minus Low
if 'High' in ddf.columns and 'Low' in ddf.columns:
        ddf['hi_lo_range'] = ddf['High'] - ddf['Low']
else:
        print("Columns 'High' and/or 'Low' not found in the DataFrame")

# Assign the result to dd_feat
dd_feat = ddf

# Convert the Dask DataFrame to a pandas DataFrame
df = dd_feat.compute()

In [13]:
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,year,Close_lag,returns,hi_lo_range
0,2013-12-02 00:00:00-05:00,34.995839,35.126735,34.760220,34.825668,2039962,0.0,0.0,A,2013,NaN,NaN,0.366515
1,2013-12-03 00:00:00-05:00,34.635858,34.799484,34.400240,34.642403,3462706,0.0,0.0,A,2013,34.825668,-0.005262,0.399244
2,2013-12-04 00:00:00-05:00,34.583484,35.231436,34.504947,35.067810,3377288,0.0,0.0,A,2013,34.642403,0.012280,0.726489
3,2013-12-05 00:00:00-05:00,34.917291,35.211813,34.766757,35.015465,2530939,0.0,0.0,A,2013,35.067810,-0.001493,0.445056
4,2013-12-06 00:00:00-05:00,35.185618,35.951373,35.185618,35.885925,4268513,0.0,0.0,A,2013,35.015465,0.024859,0.765755


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [17]:
# Write your code below.
if 'returns' in df.columns:
    df['rolling_avg_returns'] = df['returns'].rolling(window=10).mean()
else:
    print("Column 'returns' not found in the DataFrame")

# Display the resulting DataFrame
df.head()

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,ticker,year,Close_lag,returns,hi_lo_range,rolling_avg_returns
0,2013-12-02 00:00:00-05:00,34.995839,35.126735,34.760220,34.825668,2039962,0.0,0.0,A,2013,NaN,NaN,0.366515,NaN
1,2013-12-03 00:00:00-05:00,34.635858,34.799484,34.400240,34.642403,3462706,0.0,0.0,A,2013,34.825668,-0.005262,0.399244,NaN
2,2013-12-04 00:00:00-05:00,34.583484,35.231436,34.504947,35.067810,3377288,0.0,0.0,A,2013,34.642403,0.012280,0.726489,NaN
3,2013-12-05 00:00:00-05:00,34.917291,35.211813,34.766757,35.015465,2530939,0.0,0.0,A,2013,35.067810,-0.001493,0.445056,NaN
4,2013-12-06 00:00:00-05:00,35.185618,35.951373,35.185618,35.885925,4268513,0.0,0.0,A,2013,35.015465,0.024859,0.765755,NaN


In [18]:
df.to_csv('assignment1.csv')

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
No,it was not necessary. Dask is designed to handle large datasets.

+ Would it have been better to do it in Dask? Why?
Yes, Dask is better at handling large datasets as it can handle operations that are larger than memory.
(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.